In [1]:
suppressPackageStartupMessages(library(tidyverse))
suppressPackageStartupMessages(library(data.table))


In [2]:
phe_files <- list()
dfs <- list()


In [3]:
phe_files[['keep']]      <- '/oak/stanford/groups/mrivas/ukbb24983/sqc/population_stratification/ukb24983_white_british.phe'
phe_files[['Glaucoma']]  <- '/oak/stanford/groups/mrivas/ukbb24983/phenotypedata/extras/highconfidenceqc/phe/HC276.phe'
phe_files[['age_death']] <- '/oak/stanford/groups/mrivas/ukbb24983/phenotypedata/9796/24611/phe/INI40007.phe'


In [4]:
for (k in phe_files %>% names){
    df <- fread(phe_files[[k]], sep='\t', head=F)
    colnames(df) <- c('FID', 'IID', k)[1:length(colnames(df))]    
    dfs[[k]] <- df %>% mutate(
        FID = as.character(FID),
        IID = as.character(IID)    
    )
    df <- NULL
}


In [5]:
phe_files[['IOP']] <- '/oak/stanford/groups/mrivas/ukbb24983/phenotypedata/extras/iop/misc/ukb2005693_ukb37855_IOP.phe'


In [6]:
dfs[['IOP']] <- fread(
    phe_files[['IOP']],
    stringsAsFactors=FALSE,
    data.table=FALSE
) %>%
mutate(
    FID = as.character(FID),
    IID = as.character(IID)    
)


In [7]:
dfs[['Glaucoma']] %>% count(Glaucoma)


Glaucoma,n
<int>,<int>
1,479760
2,8614


In [8]:
dfs[['IOP']]      %>% inner_join(dfs[['keep']], by=c('FID', 'IID')) %>% dim() %>% print()
dfs[['Glaucoma']] %>% inner_join(dfs[['keep']], by=c('FID', 'IID')) %>% dim() %>% print()


[1] 82253     8
[1] 337151      3


In [24]:
plink_raw <- list()
for (rsid in c('rs28991009')){
    plink_raw[[rsid]] <- fread(
        paste0('ukb24983_cal_cALL_v2_hg19_', rsid, '.raw')
    ) %>%
    mutate(
        FID = as.character(FID),
        IID = as.character(IID)        
    )
}


In [18]:
read_birth_year <- function(){
    df <- fread(
        cmd=paste0(
            'cat ', 
            '/oak/stanford/groups/mrivas/ukbb24983/phenotypedata/9796/24611/download/ukb24611.tab',
            '| cut -f1,9'
        ),
        head=T, data.table=F
    )
    colnames(df) <- c('FID', 'Year_of_birth')
    df %>% mutate(FID = as.character(FID))
}

dfs[['birth_year']] <- read_birth_year()

In [25]:
merged <- dfs[['keep']] %>% 
left_join(dfs[['IOP']] %>% mutate(IOP_measured = TRUE), by=c('FID', 'IID')) %>%
replace_na(list(IOP_measured = FALSE)) %>%
inner_join(dfs[['Glaucoma']], by=c('FID', 'IID')) %>%
inner_join(dfs[['birth_year']], by=c('FID')) %>%
inner_join(dfs[['age_death']], by=c('FID', 'IID')) %>%
inner_join(
    plink_raw[['rs28991009']] %>% select(FID, IID, rs28991009_G),
    by=c('FID', 'IID')
)


In [54]:
merged %>% fwrite('private_data/IOP_pheno_dist.merged.phe', sep='\t')